In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

batch_size = 32
learning_rate = 0.01
num_epochs = 5

# MNIST Dataset Data Loader
train_dataset = datasets.MNIST(root='./data', train=True, 
                        transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data', train=False, 
                        transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
# Neural Network Modeling
class ConvNeuralNetwork(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNeuralNetwork, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out
    
# Model
model = ConvNeuralNetwork()

# Loss
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [5]:
# Train the model
for epoch in range(num_epochs):
    for i, (img, label) in enumerate(train_loader, 1):
        img, label = Variable(img), Variable(label)
        out = model(img)
        loss = criterion(out, label)
        
         # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

Epoch [1/5], Step [100/1875], Loss: 0.6108
Epoch [1/5], Step [200/1875], Loss: 0.3111
Epoch [1/5], Step [300/1875], Loss: 0.1354
Epoch [1/5], Step [400/1875], Loss: 0.2211
Epoch [1/5], Step [500/1875], Loss: 0.2044
Epoch [1/5], Step [600/1875], Loss: 0.3622
Epoch [1/5], Step [700/1875], Loss: 0.1903
Epoch [1/5], Step [800/1875], Loss: 0.2395
Epoch [1/5], Step [900/1875], Loss: 0.2468
Epoch [1/5], Step [1000/1875], Loss: 0.1655
Epoch [1/5], Step [1100/1875], Loss: 0.1430
Epoch [1/5], Step [1200/1875], Loss: 0.0651
Epoch [1/5], Step [1300/1875], Loss: 0.1476
Epoch [1/5], Step [1400/1875], Loss: 0.0585
Epoch [1/5], Step [1500/1875], Loss: 0.0335
Epoch [1/5], Step [1600/1875], Loss: 0.0419
Epoch [1/5], Step [1700/1875], Loss: 0.0522
Epoch [1/5], Step [1800/1875], Loss: 0.0541
Epoch [2/5], Step [100/1875], Loss: 0.0492
Epoch [2/5], Step [200/1875], Loss: 0.1336
Epoch [2/5], Step [300/1875], Loss: 0.0135
Epoch [2/5], Step [400/1875], Loss: 0.0179
Epoch [2/5], Step [500/1875], Loss: 0.1384
Ep

In [ ]:
# Test the model
model.eval()

correct = 0
total = 0
for img, label in test_loader:
    out = model(img)
    _, predicted = torch.max(out.data, 1)
    total += label.size(0)
    correct += (predicted == label).sum().item()

print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model_cnn.ckpt')